In [1]:
import os
import xlrd
import re
import shutil
import sys
import threading
import time

import pdfplumber

import pyzbar.pyzbar as pyzbar
import numpy as np
import cv2

In [2]:
def parse_pdf(path):
    with pdfplumber.open(path) as pdf:
#         first_page = pdf.pages[0]
#         print(first_page.chars[0])

        for page in pdf.pages:
            # 获取当前页面的全部文本信息，包括表格中的文字
            # print(page.extract_text())                        
        
            for table in page.extract_tables():
                print(table)
                for row in table:
                    print(row)
                # print('---------- 分割线 ----------')
        
        pdf.close()
        result = {'pages': pdf.pages}
        return result

In [3]:
pdf_file_name = '日用品20190322_98p99'
#     pdf_file_name = '日用品20190417_197p98'
#     pdf_file_name = '食品20190408_91p8'
path = 'data\\invoice\\{pdf_file_name}.pdf'.format(pdf_file_name=pdf_file_name)
output_path = 'data\\result\\{pdf_file_name}.txt'.format(pdf_file_name=pdf_file_name)
result = parse_pdf(path)

[['购\n买\n方', '名        称:北京慧远云程科技有限公司\n91110105306602928Y\n纳税人识别号:\n地 址、电 话:\n开户行及账号:', None, None, None, None, '密\n码\n区', '59+8>9232--*+99+333*-<9/1/-\n60/4</2*/137<6/0110*0/2<2</\n/0+9</9*<+*6*1+1+19>+130>-9\n*09>12>5+/1+/0-><>/0998+95+', None, None, None], ['货物或应税劳务、服务名称     \n*纺织产品*天石（HighRock）户外\n露营情侣左右互拼成人棉睡袋 -10度 \n右开 天蓝色\n*纺织产品*天石（HighRock）户外\n露营情侣左右互拼成人棉睡袋 -10度 \n右开 天蓝色\n合       计', None, '规格型号 \n冷杉', '单位', '数 量\n1', '单 价\n171.55', None, None, '金  额  \n171.55\n-86.22\n¥85.33', '税率\n16%\n16%', '税    额\n27.45\n-13.79\n¥13.66'], ['价税合计(大写)', None, '¥98.99\n玖拾捌圆玖角玖分 (小写)', None, None, None, None, None, None, None, None], ['销\n售\n方', '名        称:沈阳京东世纪贸易有限公司\n912101125893550575\n纳税人识别号:\n地 址、电 话:辽宁省沈阳市浑南区麦子屯590-4号102室 024-67859504\n开户行及账号:中国建设银行沈阳市中山支行 21001360008059003247', None, None, None, None, '备\n注', '订单号:90235270381', None, None, None]]
['购\n买\n方', '名        称:北京慧远云程科技有限公司\n91110105306602928Y\n纳税人识别号:\n地 址、电 话:\n开户行及账号:', None, None, None, None, '密\n码\n区', '59+8>9232--*+99+3

In [4]:
def decode(im) : 
  # Find barcodes and QR codes
  decodedObjects = pyzbar.decode(im)
 
  # Print results
  for obj in decodedObjects:
    print('Type : ', obj.type)
    print('Data : ', obj.data.decode(encoding='utf-8'),'\n')
     
  return decodedObjects
# Display barcode and QR code location  
def display(im, decodedObjects):
 
  # Loop over all decoded objects
  for decodedObject in decodedObjects: 
    points = decodedObject.polygon
 
    # If the points do not form a quad, find convex hull
    if len(points) > 4 : 
      hull = cv2.convexHull(np.array([point for point in points], dtype=np.float32))
      hull = list(map(tuple, np.squeeze(hull)))
    else : 
      hull = points;
     
    # Number of points in the convex hull
    n = len(hull)
 
    # Draw the convext hull
    for j in range(0,n):
      cv2.line(im, hull[j], hull[ (j+1) % n], (255,0,0), 3)
 
  # Display results 
  cv2.imshow("Results", im);
  cv2.waitKey(0);

In [5]:
import qrcode
img = qrcode.make('hello, 中文qrcode测试')
img.save('zbar-test.png')

In [6]:
# Read image
im = cv2.imread('zbar-test.png')

decodedObjects = decode(im)
display(im, decodedObjects)

Type :  QRCODE
Data :  hello, 中文qrcode测试 



In [7]:
import re
  
line = "IF_MIB::=Counter32: 12345\nIF_MIB::=Counter32: 1234556";
result = re.findall( r'(?<=\:\s)\d+$', line, re.M)

In [8]:
result

['12345', '1234556']

In [9]:
import fitz
from io import BytesIO
from PIL import Image

In [90]:
doc = fitz.open(path)
# png = doc[0].getPixmap(matrix=fitz.Matrix(2,2), alpha=False).getPNGdata()
# img = Image.open(BytesIO(png))
# img.save('png_from_pdf.png')
# img.show()
pm = doc[0].getPixmap(matrix=fitz.Matrix(2,2), alpha=False)
pm.writePNG('png_from_pdf.png')
im2 = cv2.imread('png_from_pdf.png')
# decodedObjects = decode(im2)
# display(im2, decodedObjects)

In [91]:
doc = fitz.open(path)
pm = doc[0].getPixmap(matrix=fitz.Matrix(2,2), alpha=False)
image_data = pm.getImageData()
# image_data = pm.getPNGData()
nparr = np.frombuffer(image_data, np.uint8)
img_np = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
# cv2.imshow("Results", img_np)
# cv2.waitKey(0);
# decodedObjects = decode(img_np)
# display(img_np, decodedObjects)

Type :  QRCODE
Data :  01,10,021001900111,23763823,85.33,20190322,70446058541415909133,E4CD, 

